For ease of running multiple experiments, the code has been extracted out of the python script and put into a jupyter notebook

Below are all settins for running experiments, please uncomment the four arguments corresponding with a certain experiment.

Make sure to run all cells. The last two cells will output the commandline scripts to run the ranklib script, and then run the evaluation afterwards.


In [1]:
import gensim
import pandas as pd
from scipy import spatial
import numpy as np
import json
import os


class args(object):
    # The following lines corresponds to: Wikipedia2Vec 2019:
    # embedding = '../src/WKN-vectors/WKN-vectors.bin'
    # ranklib_output = 'w2vconcepts'
    # score_output = 'w2vconcepts.txt'
    # embedding_type = 'wikipedia2vec_old'

    # The following lines corresponds to: Wikipedia2Vec 2015:
    # embedding = '../src/wikipedia2vec_trained_20151002/wikipedia-20151002_trained_500' 
    # ranklib_output = 'w2v2015concepts'
    # score_output = 'w2v2015concepts.txt'
    # embedding_type = 'wikipedia2vec'
    
    #The following lines corresponds to: rdf2vec
    embedding = '../src/walks_1306/model.kv'
    ranklib_output = 'rdf2vec1306concepts'
    score_output = 'rdf2vec1306concepts.txt'
    embedding_type = "rdf2vec"

    # The following lines corresponds to: rdf2vec + pagelinks
    # embedding = '../src/walks_1708/model.kv'
    # ranklib_output = 'rdf2vec1708concepts'
    # score_output = 'rdf2vec1708concepts.txt'
    # embedding_type = "rdf2vec"

    # The following lines corresponds to: complex
#     embedding = '../src/20220511-231012-dbpedia_small-complex_gpu/checkpoint_best.pt'
#     ranklib_output = 'complex2vecredconcepts'
#     score_output = 'complex2vecredconcepts.txt'
#     embedding_type = 'complex'
    

    # The following lines corresponds to: complex + pagelinks
    # embedding = '../src/20220818-151937-dbpedia_pagelinks-complex_gpu/checkpoint_best.pt'
    # ranklib_output ='complex2vecpagelinksconcepts'
    # score_output = 'complex2vecpagelinksconcepts.txt'
    # embedding_type = 'complex'
    

    
    
    # Use any of the following linkers, or other linkers in the /Data directory: 

    linker = "../Data/geeer_ready.csv"
    #linker = "../Data/geeer_annotated_radboud.csv"
    #linker = "../Data/geeer_annotated_ELQ.csv"
    #linker = "../Data/geeer_annotated_total.csv"



    dbpedia_input = "../src/DBpedia-Entity/runs/v2/bm25f-ca_v2.run"
    def __init__(self, i_var):
        self.i_var = i_var

path_to_dbpedia = "../src/DBpedia-Entity"

In [2]:
print(gensim.__version__)

4.3.1


def entity_converter(word, reverse = False, nospace = True):
    # Input: an entity string in dbpedia format
    # Output: string in wikipedia2vec format or readible format
    # Getting entities in the right format
    if args.embedding_type == 'rdf2vec':
            word = word.replace("<dbpedia:", "http://dbpedia.org/resource/")
            word = word.replace(">", "")
            return(word)
    elif args.embedding_type == 'complex':
        word = word.replace("dbpedia:", "http://dbpedia.org/resource/")
        return(word)
    elif reverse:
        word = word.replace("<dbpedia:", "")
        word = word.replace(">", "")
        if nospace:
            return word
        else:
            word = word.replace("_", " ")
            return word
    else:
        word = word.replace("<dbpedia:", "ENTITY/")
        word = word.replace(">", "")
        return word

In [3]:
def entity_lookup(tag, model, tagme = False):
    # Input: entity string
    # Output: True wikipedia2vec format
    # Looking up the embedding of entities, returning [] when entity not in corpus
    if tagme and args.embedding_type == 'rdf2vec':
        tag = tag.replace('ENTITY/', 'http://dbpedia.org/resource/')
    elif tagme and args.embedding_type == 'complex':
        tag = tag.replace('ENTITY/', '<http://dbpedia.org/resource/')
        tag = tag + '>'
    elif tagme and args.embedding_type == 'wikipedia2vec':
        tag = tag.replace('ENTITY/', '')
        tag = tag.replace('_', ' ')
    elif tagme and args.embedding_type == 'wikipedia2vec_old':
        tag = tag
    elif tag in redirect_dict:
        tag = redirect_dict[tag]
    else:
        try: 
            backup = entity_converter(redirect_dict[tag])
        except KeyError as e:
            backup = ''
        tag = entity_converter(tag)
    if args.embedding_type == 'wikipedia2vec': 
        if model.get_entity(tag):
            return model.get_entity_vector(tag)
        elif model.get_entity(backup):
            #print(tag, backup)
            return model.get_entity_vector(backup)
        else:
            return []
    if args.embedding_type == 'rdf2vec' or args.embedding_type == 'wikipedia2vec_old':
        #if tag in model.vocab:
        if tag in model.key_to_index:
            return model[tag]
        # elif backup in model.vocab:
        #     #print(tag, backup)
        #     return model[backup]
        else:
            return []
    if args.embedding_type == 'complex': 
        #if tag in model.dataset.entity_ids():
        if tag in entity_ids_dict:
            #ent_index = model.dataset.entity_ids().index(tag)
            ent_index = entity_ids_dict[tag]
            model_tag = model.get_s_embedder().embed(torch.Tensor([ent_index]).long())
            return model_tag
        elif backup in entity_ids_dict:
            ent_index = entity_ids_dict[backup]
            model_tag = model.get_s_embedder().embed(torch.Tensor([ent_index]).long())
            #print(tag, backup)

            return model_tag
        else:
            return []
    else:
        return []


In [4]:
def entity_converter(word, reverse = False, nospace = True):
    # Input: an entity string in dbpedia format
    # Output: string in wikipedia2vec format or readible format
    # Getting entities in the right format
    if args.embedding_type == 'rdf2vec':
        word = word.replace("<dbpedia:", "http://dbpedia.org/resource/")
        word = word.replace(">", "")
        return(word)
    elif args.embedding_type == 'complex':
        word = word.replace("dbpedia:", "http://dbpedia.org/resource/")
        return(word)
    elif args.embedding_type == 'wikipedia2vec':
        word = word.replace("<dbpedia:", "")
        word = word.replace(">", "")
        word = word.replace("_", " ")
        return(word)
    elif args.embedding_type == 'wikipedia2vec_old':
        word = word.replace("<dbpedia:", "ENTITY/")
        word = word.replace(">", "")
        return word
    elif reverse:
        word = word.replace("<dbpedia:", "")
        word = word.replace(">", "")
        if nospace:
            return word
        else:
            word = word.replace("_", " ")
            return word
    else:
        word = word.replace("<dbpedia:", "ENTITY/")
        word = word.replace(">", "")
        return word

In [5]:
# from timeit import default_timer as timer

# start = timer()
# ent_index = model.dataset.entity_ids().index('')
# model.get_s_embedder().embed(torch.Tensor([1]).long())
# end = timer()
# print(end - start)

In [6]:
if args.embedding_type == 'complex':
    import torch
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
def ranking_feature(query, entity, dist = "cosine", conf = True):
    # Input: a query-entity pair to score, a distance function and if we want to use confidence scores
    # Output: query-entity based score

    # getting all the linked entities to the query
    #a_ent = pd.DataFrame(confidence.loc[confidence['query_id'] == query])
    if query not in confidence:
    #if len(a_ent)== 0:
        # if no linked queries, return 0
        return 0
    else:
        total = 0
        a_ent = confidence[query]
        for row in a_ent:
        #for index, row in a_ent.iterrows():
            score = row['confidence']
            #score = row['score']
            ent1 = entity_lookup(entity, model)
            ent2 = entity_lookup(row['tag'], model, tagme = True)
            #ent2 = entity_lookup(row['entity'])
            if len(ent1) == 0 or len(ent2) == 0:
                continue
            else:
                if dist == 'fjaccard':
                    dist = 1-np.minimum(ent1,ent2).sum()/np.maximum(ent1,ent2).sum()
                elif args.embedding_type == 'complex':
                    dist = cos(ent1, ent2).detach().numpy()[0]
                else:
                    dist = 1 - spatial.distance.cosine(ent1, ent2)
                if conf:
                    score = row['confidence']
                    total += score*dist
                else:
                    total += dist
        return total

In [7]:
def to_print_format(queries, filepath):
    # input: A list of queries of which we want the results
    # output: Ranklib ready format of results
    f = open(filepath, "w")
    for q in queries:
        entities = new_df.loc[(new_df['query_id'] == q)]
        for index, row in entities.iterrows():
            printstring = str(int(row['rel'])) + ' qid:' + row['query_id'] + " 1:" + str(row['embedding_score']) + " 2:" + str(row['fsdm_score']) + " # " + row['tag']
            print(printstring, file = f)
    f.close()

In [8]:
def to_print_format_dict(queries, filepath):
    # input: A list of queries of which we want the results
    # output: Ranklib ready format of results
    f = open(filepath, "w")
    for q in queries:
        entities = rerank[q]
        for ent in entities.keys():
            printstring = str(int(entities[ent]['rel'])) + ' qid:' + q + " 1:" + str(entities[ent]['score']) + " 2:" + str(entities[ent]['fsdm_score']) + " # " + ent
            print(printstring, file = f)
    f.close()

In [10]:
print("Loading auxilary files")
# Loading the file to rerank
# Note, if an error is given here when loading a different file to rerank, try changing the seperator to '\t'

qrels_path = path_to_dbpedia + '/collection/v2/qrels-v2.txt'
qrels = {}
with open(qrels_path) as f:
    for line in f:
        terms = line.split()
        if terms[0] not in qrels:
            qrels[terms[0]] = {}
        qrels[terms[0]][terms[2]] = terms[3]



rerank_path = args.dbpedia_input
#rerank =  pd.read_csv(rerank_path, sep='\s+', names = ['query_id', 'x1', 'tag', 'rang', 'fsdm_score', 'x2'])
rerank = {}
with open(rerank_path) as f:
    for line in f:
        terms = line.split()
        if terms[0] not in rerank:
            rerank[terms[0]] = {}
        if terms[2] in qrels[terms[0]]:
            rel = qrels[terms[0]][terms[2]]
        else:
            rel = 0
        rerank[terms[0]][terms[2]] = {'fsdm_score' : terms[4], 'rel' : rel}


# Loading linked entities
#confidence = pd.read_csv(args.linker)


confidence = {}
with open(args.linker) as f:
    f.readline()
    for line in f:
        terms = line.strip().split(',')
        if terms[1] not in confidence:
            confidence[terms[1]] = []
        try: 
            confidence[terms[1]].append({'tag' : terms[2], 'confidence' : float(terms[3])})
        except ValueError as e:
            conf = terms[-1]
            tag = ','.join(terms[2:-1])
            confidence[terms[1]].append({'tag' : tag, 'confidence' : float(conf)})


# Loading auxilary files
#qrels_path = path_to_dbpedia + '/collection/v2/qrels-v2.txt'
#qrels = pd.read_csv(qrels_path, sep='\t',names = ['query_id', '', 'tag', 'rel'])
queries_path = path_to_dbpedia + '/collection/v2/queries-v2.txt'
queries = pd.read_csv(queries_path, sep='\t',names = ['query_id', 'query'])

# Loading previously computed redirects
#df = pd.read_csv('/store/usr/gerritse/results_dict/wikipedia_redirect.csv')
df = pd.read_csv('../Data/wikipedia_redirect.csv')

# Getting the data ready for further processing in RankLib
folds_path = path_to_dbpedia + "/collection/v2/folds/all_queries.json"
with open(folds_path, 'r') as read_file:
    data = json.load(read_file)

Loading auxilary files


In [11]:
print("Loading embeddings")
# Loading the model with a Gensim keyedvector
if args.embedding_type == 'rdf2vec' or args.embedding_type ==  'wikipedia2vec_old': 
    model = gensim.models.KeyedVectors.load(args.embedding, mmap='r')
    
elif args.embedding_type == 'wikipedia2vec':
    from wikipedia2vec import Wikipedia2Vec
    model = Wikipedia2Vec.load(args.embedding)

elif args.embedding_type == 'complex': 
    import torch
    from kge import Dataset
    from kge.model import KgeModel
    from kge.util.io import load_checkpoint
    checkpoint = load_checkpoint(args.embedding)
    if args.ranklib_output == 'complex2vecpagelinksred':
        dataset = Dataset.create(checkpoint['config'],folder = '../src/dbpedia_kge_pagelinks')
    else:
        dataset = Dataset.create(checkpoint['config'],folder = '../src/dbpedia_kge_small')

    model = KgeModel.create_from(checkpoint, dataset)
    entity_ids = model.dataset.entity_ids()
    entity_ids_dict = {}
    for i, e in enumerate(entity_ids):
        entity_ids_dict[e] = i

    import torch
    cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

redirect_dict = {}
for index, tags in df.iterrows():
    redirect_dict[tags['original']] = tags['redirect']



Loading embeddings


FileNotFoundError: [Errno 2] No such file or directory: '../src/walks_1306/model.kv.vectors.npy'

In [16]:
#redirect_dict

In [17]:
"a" in model.key_to_index

True

In [19]:
entity_lookup(entity_converter('<dbpedia:Amsterdam>'), model)

memmap([ 0.0267, -0.6669, -0.577 ,  0.1185,  0.3543,  0.3663,  0.6281,
        -0.6389,  0.0808, -0.2223,  0.3297, -0.1004, -0.9479,  0.4323,
         0.4797, -0.0581,  0.5623, -1.7071,  0.7628,  0.5647,  0.3995,
         0.7988, -0.3355,  0.2411, -0.9855, -0.8247, -0.6165, -1.2618,
        -0.7464,  0.084 , -0.5706, -0.1865, -0.4999, -0.4771,  0.6964,
        -0.3774,  0.1878,  0.019 , -0.5471,  1.0492,  0.2103, -0.7212,
         0.2763, -0.0965, -0.0764, -1.1733, -0.9302,  0.2558,  0.1496,
         0.5607, -0.2725, -0.0023,  0.6871,  0.4769, -0.6926, -0.779 ,
        -0.0532, -0.1834, -0.1065,  0.4034, -0.1091,  0.491 , -0.5935,
         0.4969, -0.284 ,  1.0307, -0.0463,  0.1828, -0.4415, -0.5461,
         0.4013, -0.2757,  0.1984, -0.914 ,  0.1912, -0.2991, -0.7957,
         1.1819,  0.2321, -0.2361, -0.6642, -0.1363,  0.538 , -0.6315,
        -0.4699,  0.207 , -0.0192, -0.0614,  0.4145,  0.8567, -0.4035,
         1.2693,  0.0799, -0.2084,  0.7934, -1.7709, -0.2036, -0.3543,
      

In [20]:
entity_converter('<dbpedia:Dutch_East_India_Company>')

'ENTITY/Dutch_East_India_Company'

In [21]:
entity_lookup(entity_converter('<dbpedia:Dutch_East_India_Company>'), model)

memmap([-0.0214, -0.6349, -0.6178,  0.2468,  0.4132,  0.1446,  0.1607,
         0.6023, -0.2738, -0.2597,  0.408 , -1.471 , -1.1823,  0.0815,
        -0.2833, -0.3059,  0.115 , -0.8741,  0.2282,  0.8581,  0.4545,
        -0.3629, -1.0791, -0.4466,  0.2917, -1.4452, -0.0279, -2.4559,
        -0.8016,  0.2948, -0.9418, -1.21  , -0.4692, -1.0293, -0.069 ,
         1.1049, -0.2965,  1.3669, -0.6884,  1.013 , -0.3521, -1.0403,
         0.6659,  1.0847, -0.2916, -1.4498, -1.1323,  0.8031, -0.5639,
        -0.198 , -1.0912, -1.0175,  1.1082,  1.2766,  0.8299, -0.6207,
        -0.4435,  0.4403,  0.228 ,  0.1291,  0.4034,  0.6301, -1.0996,
        -0.335 , -0.1544,  0.9508,  0.5949, -0.6929,  0.4635, -0.0732,
        -0.018 , -0.9697, -0.2717, -0.2482,  0.6188, -0.6333, -1.5828,
         0.8138, -0.3467, -0.9016, -0.238 ,  0.0231,  0.0422, -0.1829,
        -1.1991,  0.9969,  0.2114, -0.9918, -0.6421, -0.4299,  0.4223,
         0.7993,  0.6424,  1.192 ,  0.869 , -1.6395,  0.9886, -0.7676,
      

from timeit import default_timer as timer

start = timer()

ent1 = entity_lookup(entity_converter('<dbpedia:Nijmegen>'), model)
end = timer()
print(end - start)
ent2 = entity_lookup(entity_converter('<dbpedia:Amsterdam>'), model)
end = timer()
print(end - start)
#cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
#cos(ent1, ent2).detach().numpy()[0]
end = timer()
print(end - start)
#spatial.distance.cosine(ent1, ent2)

In [25]:
# Merging the qrels and rerank for scoring
# new_df = pd.merge(qrels[['query_id','tag','rel']], rerank[['query_id','tag','fsdm_score']],  how='right', left_on=['query_id','tag'], right_on = ['query_id','tag'])
# new_df = new_df.fillna(value = 0)
# new_df['rel'].isna().sum()

# Scoring everything (might take a while)
test_x = []

f = open(args.score_output, 'w')
#N = len(new_df)
#l = 50
#m = (N/l)

print("Ranking entities:")
#for index, row in new_df.iterrows():
for query_id in rerank.keys():
    print(query_id)
    for tag in rerank[query_id].keys():        
        #i = int(index/m)
        #percentage = (100 * index) // N
        #print('[' + '-'*i + ' '*(l-1-i)+ '] ' + str(percentage) +"%", flush = True, end ='\r')
        #print(row['query_id'],row['tag'])
        query_based_score = ranking_feature(query_id, tag, conf = True)
        #print(query_id, tag, query_based_score)
        test_x.append(query_based_score)
        #print(rerank[query_id][tag])
        rerank[query_id][tag]['score'] = query_based_score
        # start = timer()
        print(" ".join([query_id, tag, str(query_based_score)] ), file=f)
        # #print(" ".join([row['query_id'], row['tag'], str(query_based_score)] ))
        # #if i > 1: 


    
# new_df['embedding_score'] = test_x
# print("\n")


Ranking entities:
INEX_LD-2012307
INEX_LD-2012321
INEX_LD-2012367
INEX_LD-20120122
INEX_LD-20120422
INEX_LD-20120522
INEX_LD-2012349
INEX_LD-2012390
INEX_LD-2009061
INEX_LD-2009039
INEX_LD-2012381
INEX_LD-2010037
INEX_LD-2009063
INEX_LD-2012389
INEX_LD-2010043
INEX_LD-2010057
INEX_LD-2012361
INEX_LD-2012337
INEX_LD-2009115
INEX_LD-2012363
INEX_LD-20120312
INEX_LD-2012375
INEX_LD-2012305
INEX_LD-20120232
INEX_LD-2012331
INEX_LD-20120121
INEX_LD-2012357
INEX_LD-20120231
INEX_LD-20120512
INEX_LD-20120332
INEX_LD-2010004
INEX_LD-20120212
INEX_LD-2009111
INEX_LD-2012311
INEX_LD-2012333
INEX_LD-2012319
INEX_LD-2012351
INEX_LD-20120412
INEX_LD-2010069
INEX_LD-20120222
INEX_LD-2010014
INEX_LD-2012339
INEX_LD-2010100
INEX_LD-2012335
INEX_LD-2012365
INEX_LD-2012359
INEX_LD-2012355
INEX_LD-20120311
INEX_LD-2009074
INEX_LD-20120111
INEX_LD-2009096
INEX_LD-20120531
INEX_LD-20120511
INEX_LD-2012373
INEX_LD-20120411
INEX_LD-2012309
INEX_LD-2009062
INEX_LD-20120112
INEX_LD-2012301
INEX_LD-2012341
INEX

In [28]:

print("Saving files for Ranklib:")
for i in range(5):
    query_fold = data[str(i)]
    folder = args.ranklib_output + "/Fold" +str(i+1)

    if not os.path.exists(folder):
        os.makedirs(folder)
    
    testpath = folder + "/test.txt"
    print("Now writing test for fold " + str(i+1))
    to_print_format_dict(query_fold['testing'], testpath)

    trainpath = folder + "/train.txt"
    print("Now writing train for fold " + str(i+1))
    test_print = to_print_format_dict(query_fold['training'], trainpath)


Saving files for Ranklib:
Now writing test for fold 1
Now writing train for fold 1
Now writing test for fold 2
Now writing train for fold 2
Now writing test for fold 3
Now writing train for fold 3
Now writing test for fold 4
Now writing train for fold 4
Now writing test for fold 5
Now writing train for fold 5


In [29]:
args.ranklib_output

'w2vELQ2'

In [30]:
print(f"bash Code/train_ranklib.sh {args.ranklib_output}")
print(f"bash Code/score_ranklib.sh {args.ranklib_output}")

bash Code/train_ranklib.sh w2vELQ2
bash Code/score_ranklib.sh w2vELQ2


In [31]:

print(f"sh eval_to_tex.sh ../{args.ranklib_output}/embed.run")

sh eval_to_tex.sh ../w2vELQ2/embed.run
